# Необходимо поднять RAG-pipeline на базе конспектов.

В качестве базы конспектов были выбраны викиконспекты от университета ИТМО по предмету "Алгоритмы и структуры данных". По информации об их crawling'е обратите внимание на ноутбук `DBFiller.ipynb`.

## Подготовка

In [1]:
# Run if exists
!unzip -q chroma_db.zip

unzip:  cannot find or open chroma_db.zip, chroma_db.zip.zip or chroma_db.zip.ZIP.


Установим зависимости

In [2]:
%pip install requests pypdf tiktoken
%pip install langchain langchain-chroma chromadb
%pip install ollama langchain-community
%pip install docker

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Запустим локальную модель для начала

In [ ]:
!/usr/bin/bash up_ollama.sh
import docker


def is_container_running(container_name: str):
    """Verify the status of a container by it's name

    :param container_name: the name of the container
    :return: boolean or None
    """
    RUNNING = "running"
    docker_client = docker.from_env()
    try:
        container = docker_client.containers.get(container_name)
    except docker.errors.NotFound as exc:
        print(f"Check container name!\n{exc.explanation}")
    else:
        container_state = container.attrs["State"]
        return container_state["Status"] == RUNNING

import time
while not is_container_running("ollama"):
    time.sleep(10)

In [ ]:
!docker exec ollama ollama run llama3

In [1]:
import os

model_name = "llama3"
url = "http://0.0.0.0:11434" 

## Подготовка базы данных

In [2]:
def normalize(text):
    return (
        text
        .replace('«','')
        .replace('»','')
        .replace('"','')
        .replace('<','')
        .replace('>','')
    )


In [3]:
with open('./text_db/formal_languages.txt',encoding='utf-8') as f:
    formal = ''.join(f.readlines())

formal = [normalize(x) for x in formal.split('-----')]

In [4]:
reduced_fragments = set(formal)

## Подготовка модели

In [7]:
import requests

def get_ollama_embeddings(text):
	response = requests.post(f"{url}/api/embeddings", json={
		"model": model_name,
		"prompt": text
	})

	return response.json().get("embedding")


In [8]:
from langchain_chroma import Chroma
from chromadb import Documents, EmbeddingFunction, Embeddings
from chromadb.types import Vector

class MyEmbeddingFunction(EmbeddingFunction):
    @staticmethod
    def embed_documents(input: Documents) -> Embeddings:
        
        return [get_ollama_embeddings(text) for text in input]
    
    @staticmethod
    def embed_query(query: str) -> Vector:
        return get_ollama_embeddings(query)


In [9]:
import chromadb
from chromadb.errors import InvalidDimensionException

db_dirname = "./chromadb"
if not os.path.exists(db_dirname):
    chromadb.api.client.SharedSystemClient.clear_system_cache()

    try:
        db = Chroma.from_texts(list(reduced_fragments), embedding=MyEmbeddingFunction, persist_directory=db_dirname)
    except InvalidDimensionException:
        Chroma().delete_collection()
        db = Chroma.from_texts(list(reduced_fragments), embedding=MyEmbeddingFunction, persist_directory=db_dirname)
else:
    db = Chroma(embedding_function=MyEmbeddingFunction, persist_directory=db_dirname)



## Настройка RAG-конвейера

In [10]:
import langchain.chains
import langchain.prompts
from ollama import Client
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnablePassthrough

import typing as tp
from langchain_core.language_models.llms import LLM

class CustomLlama(LLM):
    def _call(self, prompt, **kwargs):
        client = Client(
            host=url
        )
        response = client.chat(model=model_name, messages=[
            {
                'role': 'user',
                'content': prompt,
            },
        ])

        return response.message.content

    @property
    def _identifying_params(self) -> tp.Dict[str, tp.Any]:
        """Return a dictionary of identifying parameters."""
        return {
            "model_name": "CustomChatModel",
        }

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "custom"

ollama_llm = CustomLlama()

prompt = """
Ты — русскоязычный ассистент. Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста. Если в тексте нет нужной информации, напиши, что не нашёл ответ на заданный вопрос. Выведи только
краткий ответ, не надо пояснительного текста. Отвечай только на поставленный вопрос. Пожалуйста, отвечай на вопросы только на русском языке.
Текст:
-----
{context}
-----
Вопрос:
{question}"""

prompt = langchain.prompts.PromptTemplate(
    template=prompt, input_variables=["context", "question"]
)

def join_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Тестирование

In [11]:
question = "Дай определение ДКА"
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})
res = retriever.invoke(question)

Number of requested results 20 is greater than number of elements in index 17, updating n_results = 17


In [12]:
chain = (
    {"context": retriever | join_docs, "question": RunnablePassthrough()}
    | prompt
    | ollama_llm
    | StrOutputParser()
)

In [13]:
result = chain.invoke(question)
print(result)

Number of requested results 20 is greater than number of elements in index 17, updating n_results = 17


ДКА (Deterministic Finite Automaton) - это автомат, который принимает входную строку символов и на основе этой строки производит вывод. В отличие от НФА (Nondeterministic Finite Automaton), у ДКА есть конкретный переход из любого состояния в любое другое состояние, если введен соответствующий символ.

ДКА имеет следующие компоненты:

* Q - множество состояний
* Σ - алфавит
* δ (дельта) - функция перехода, которая принимает два аргумента: текущее состояние и символ из алфавита, и возвращает следующее состояние
* q0 - начальное состояние
* F - множество терминальных состояний

ДКА работает следующим образом:

1. Начинается с начального состояния q0.
2. Вводится символ из алфавита.
3. Автомат переходит в новое состояние, определенное функцией δ по текущему состоянии и введенному символу.
4. Процесс повторяется до тех пор, пока не будет достигнуто терминальное состояние (например, когда автомат принимает пустую строку).
5. Выводом является последовательность состояний, которые были пройден

In [ ]:
question = "Напиши определение изоморфизма автоматов"
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})
res = retriever.invoke(question)

chain = (
    {"context": retriever | join_docs, "question": RunnablePassthrough()}
    | prompt
    | ollama_llm
    | StrOutputParser()
)

In [ ]:
result = chain.invoke(question)
print(result)

Number of requested results 20 is greater than number of elements in index 17, updating n_results = 17


Автоматы называются '''изоморфными''' (англ. ''isomorphic''), если существует [[Отображения | биекция]] между их вершинами такая, что:

1. Терминальные состояния соответствуют терминальным;
2. Начальные состояния соответствуют начальным;
3. Все переходы сохраняются: из одного состояния в другое может перейти только то же самое состояние в другом автомате, если существуют оба перехода.

В других словах, изоморфизм автоматов означает, что есть биективное отображение между вершинами (состояниями) первого автомата на вершины второго автомата, при котором терминальные состояния остаются терминальными, начальные — начальными, и все переходы сохраняются.


In [ ]:
question = "Кто такие покемоны?"
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})
res = retriever.invoke(question)

chain = (
    {"context": retriever | join_docs, "question": RunnablePassthrough()}
    | prompt
    | ollama_llm
    | StrOutputParser()
)

In [20]:
result = chain.invoke(q)
print(result)

Number of requested results 20 is greater than number of elements in index 17, updating n_results = 17


Я не знаю, кто такие покемоны. В моей базе знаний нет информации об этом тематике. Моя задача - помочь с вопросами по теории формальных языков и грамматиках. Если у вас есть какие-то вопросы или проблемы в этой области, я готов помочь!


## Вывод

За исключением того, что модель ошибается в написании и формировании русских слов, она даёт правильные ответы на поставленные вопросы.